In [16]:
# write and read from the FSM and compare with LabVIEW vi
import nidaqmx
import numpy as np

conversion = {'x': 8.54, 'y': 6.19}

def microm_to_volts(pos_in_microm, axis):
    return pos_in_microm / conversion[axis]

def volts_to_microm(pos_in_volts, axis):
    return pos_in_volts * conversion[axis]

def read_position():
    with nidaqmx.Task() as fsm_task:
        fsm_task.ai_channels.add_ai_voltage_chan('/Dev1/ai1', 'FSM x axis')
        fsm_task.ai_channels.add_ai_voltage_chan('/Dev1/ai0', 'FSM y axis')
        curr_x, curr_y = fsm_task.read()
        curr_x = volts_to_microm(curr_x, 'x')
        curr_y = volts_to_microm(curr_y, 'y')
    return curr_x, curr_y

def calc_trajectory():
    pass

def go_to_position(x, y):
    pass

read_position()

(-7.169819558388406, 4.795168790164483)

In [14]:
import time
# go to a position --> use smooth sinusoidal sweeping from start to end position
x_terminal_out = '/Dev1/ao1'
y_terminal_out = '/Dev1/ao0'
new_pos = (0, 0)   # um
scan_freq = 1000
n_pts = 200

# read current position
curr_pos = read_position()

# make sine-like profile for sweeping the values from one position to the next
delta_pos = microm_to_volts(new_y, 'y') - curr_pos
x_sine = np.linspace(-np.pi/2, np.pi/2, n_pts)
vsweep_array = (np.sin(x_sine)+1)/2*delta_pos + curr_pos

# scan the FSM until the desired position
with nidaqmx.Task() as task:
    output_volt = task.ao_channels.add_ao_voltage_chan(y_terminal_out, 'FSM y axis', min_val=-3.5, max_val=3.5)
    #output_volt.ao_term_cfg = nidaqmx.constants.TerminalConfiguration.DEFAULT
    
#     task.timing.samp_timing_type = nidaqmx.constants.SampleTimingType.SAMPLE_CLOCK
#     task.timing.samp_quant_samp_mode = nidaqmx.constants.AcquisitionType.FINITE
#     task.timing.samp_quant_samp_per_channel = n_pts
#     task.timing.samp_clk_rate = scan_freq
#     task.timing.samp_clk_active_edge = nidaqmx.constants.Edge.RISING
    #task.timing.samp_clk_src = 
    
    task.write(vsweep_array, auto_start=True, timeout=5)
    task.wait_until_done(timeout=5)

# read the new position
time.sleep(.5)
with nidaqmx.Task() as y_task:
    y_task.ai_channels.add_ai_voltage_chan(y_terminal)
    curr_pos = y_task.read(number_of_samples_per_channel=8)
    curr_pos = np.mean(curr_pos)
    print('y: volts:', curr_pos, 'um', volts_to_microm(curr_pos, 'y'))

Old y: volts: 0.1492013435944409 um 0.923810542976022
y: volts: -0.1742921456912046 um -1.0791653605017995


In [ ]:
# scan_size = 10   #um
# mesh_size = 100  #number of points per line scan
# scan_rate = 100  #Hz
# terminal = "/Dev1/PFI1"

#pts = np.linspace(-scan_size/2, scan_size/2, 100)

# make array of X values to scan at each new Y position
x_scan = np.linspace(min_x, max_x, mesh_size)
y_scan = np.linspace(min_y, max_y, mesh_size)

# move to Y value and minimum value of X array

# start clock, APD counter, AO task for FSM
# use clock to move FSM to next X position (rising edge), read APD counts (falling edge)
# do I need to read FSM position?
# read all buffered APD counts at once
# plot 1D scan
# append to 2D scan and plot